# Web Scraping Drudgereport
- This notebook is a tutorial on how to scrape the Drudgereport website with Python. 

In [1]:
# import all modules that will be used in our scraper

from requests import get 
from bs4 import BeautifulSoup
from time import sleep
import re
from random import randint 
from warnings import warn
from time import time
from IPython.core.display import display
import numpy as np
import pandas as pd
from datetime import datetime

/tmp/ipykernel_612014/963368025.py:10: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display


In [2]:
# request the contents of the page we're scraping

results = get('https://www.drudgereport.com/')

In [6]:
# make the content we grabbed easy to read

html_soup = BeautifulSoup(results.text, 'html.parser')

In [8]:
html_soup

 <html>
<head>
<meta content="upgrade-insecure-requests" http-equiv="Content-Security-Policy"/>
<title>DRUDGE REPORT 2024®</title>
<!-- eProof, DR Headline Updater -->
<script src="//cdnjs.cloudflare.com/ajax/libs/jquery/3.4.1/jquery.min.js" type="text/javascript"></script>
<script src="//eproof.drudgereport.com/dr.js" type="text/javascript"></script>
<!-- eProof.com end -->
<script src="/js/cookie_filter_1.0.1.js" type="text/javascript"></script>
<script type="text/javascript">
<!-- 
var dr_do_bt = 1; var img = new Image(), url = "/204.png", container = document.getElementById("div-204");
img.onload = function () { container.appendChild(img); };
img.src = url;
var timer = setInterval("__drudge__321__autoRefresh()", 1000 * 33 * 3);
function __drudge__321__autoRefresh(){self.location.reload(true);}
(function () {
    var __oldClearInterval = clearInterval;

    clearInterval = function (arg) {
        if(arg == timer) {
                console.log("clearInterval of TIMER intercepted! ("

In [10]:
# get the macro-container containing the posts we want

# post = html_soup.find_all('li', class_= 'result-row')

post = html_soup.find_all('a')

print(type(post)) # check I got a ResultSet
print(len(post)) # check I got 120 

<class 'bs4.element.ResultSet'>
220


In [11]:
# grab the first post

post_one = post[0]

In [12]:
print(post_one)

<a href="https://www.cnn.com/2023/06/09/politics/trump-tape-didnt-declassify-secret-information/index.html" nopush=""><font color="red">Ex-president admits on tape he didn't declassify 'secret information'...</font></a>


In [13]:
post

[<a href="https://www.cnn.com/2023/06/09/politics/trump-tape-didnt-declassify-secret-information/index.html" nopush=""><font color="red">Ex-president admits on tape he didn't declassify 'secret information'...</font></a>,
 <a href="https://www.washingtonpost.com/national-security/2023/06/09/trump-tape-classified-documents/" nopush=""><font color="red"><i>Incriminating statements on Iran papers...</i></font></a>,
 <a href="https://www.politico.com/news/2023/06/09/trump-loses-two-lawyers-just-hours-after-being-indicted-00101263" nopush=""><font color="red">TOP LAWYERS ABRUPTLY RESIGN...</font></a>,
 <a href="https://www.wsj.com/articles/trump-associate-also-indicted-in-mar-a-lago-documents-case-759cbb17">Aide Seen Moving Boxes Also Hit With Indictment...</a>,
 <a href="https://www.msn.com/en-us/news/politics/trump-case-presents-extraordinary-test-of-merrick-garland-s-integrity/ar-AA1cl3et">Extraordinary test of Garland's integrity...</a>,
 <a href="https://www.cnbc.com/2023/06/09/trump-i

In [15]:
#grab the text of the first post

post_one_text = post_one.text

post_one_text

"Ex-president admits on tape he didn't declassify 'secret information'..."

In [16]:
post_one_text

"Ex-president admits on tape he didn't declassify 'secret information'..."

In [17]:
# grab the link

post_one_link = post_one['href']
post_one_link

'https://www.cnn.com/2023/06/09/politics/trump-tape-didnt-declassify-secret-information/index.html'

Goals:
- Story = Story or Photo
- Story arrives
- Story departs
- Story location changes
- Story text changes
- Story Size or Color changes
- Story Duration

- Constant Link arrives
- Constant Link departs

Locations:
- Top left
- Top Center
- Top Right
- Bottom Left
- Bottom Center
- Bottom Right

<!-- TOP LEFT STARTS HERE--><tt><b><br/><br/>
<!-- MAIN HEADLINE--><font face="ARIAL,VERDANA,HELVETICA"><font size="+7"><img height="400" src="https://pbs.twimg.com/media/FItdwvBWUAMSGf8?format=png&amp;name=small"/><br/><a href="https://www.tmz.com/2022/01/09/bob-saget-dead-dies-hotel-comedian-full-house/" nopush="">SAGET DEAD IN ORLANDO HOTEL ROOM</a><br/>
<a href="https://www.orlandosentinel.com/news/breaking-news/os-ne-bob-saget-dead-20220110-n47pzod2hfeg3fu2mb25lnzw2a-story.html" nopush=""><i>CAUSE UNKNOWN</i></a><br/>
<a href="https://www.the-sun.com/entertainment/4433230/bob-saget-found-dead-ritz-carlton-orlando/" nopush="">FOUND BY HOUSEKEEPING AFTER MISSING CHECKOUT</a>
<a href="https://www.drudgereport.com"><img border="0" height="85" src="/i/logo9.gif" width="610"/></a>
</center><br/>
</b></tt>
<center><table cellpadding="3" width="100%"><tr>
<td align="LEFT" valign="TOP" width="33%"><tt><b>
<img src="https://th.bing.com/th/id/OIP.sg-ad90w-6C9BMCWpHWGpQHaKX?w=125&amp;h=180&amp;c=7&amp;r=0&amp;o=5&amp;dpr=1.1&amp;pid=1.7" width="200"/>
<br/><br/>
<!--    L I NKS    FI RS T    C O LU MN-->
<a href="http://www.thepaperboy.com/uk/front-pages.cfm">FRONT PAGES UK</a> <a href="https://news.sky.com/topic/newspapers-5993">THE PAPERS</a><br/>


<a href="https://www.zerohedge.com/">ZEROHEDGE</a>
</b></tt>
</td>

Begin Center Column
Begin Web Race Stats
<td align="center" valign="top" width="3"><div style="width:1px;background-color:#C0C0C0;margin-left:1px;margin-right:1px;height:2500px;"></div></td>
<td align="LEFT" valign="TOP" width="33%"><tt><b>
<a href="https://pbs.twimg.com/media/FItqn2GXsAMFvsg?format=jpg&amp;name=medium">
<img border="0" src="https://pbs.twimg.com/media/FItqn2GXsAMFvsg?format=jpg&amp;name=medium" width="200"/></a>
<hr/>
End Web Race Stats

Center Column Posts Begin
<a href="https://www.bnnbloomberg.ca/bitcoin-is-off-to-its-worst-annual-start-since-the-dawn-of-crypto-1.1705389" nopush=""><font color="red">Bitcoin Off to Worst Start Since Dawn of Crypto...</font></a>
<br/><br/>

<a href="https://www.the-sun.com/news/4427364/western-troops-ill-wuhan-months-before-covid-outbreak/">Western troops sick in Wuhan months before Covid outbreak officially began...</a>
<br/><br/>
Center Column Posts End
<a href="https://www.arcgis.com/apps/opsdashboard/index.html#/bda7594740fd40299423467b48e9ecf6">WORLD SICK MAP...</a>
<hr/>
<!-- Granite3 -->
<!-- /218243714/DR-right_300x250_1 -->
<div id="div-gpt-ad-1567201323104-0">
<script>
    googletag.cmd.push(function() { googletag.display('div-gpt-ad-1567201323104-0'); });
  </script>
</div>
<div id="div-gpt-ad-1564811240422-0"></div>
<hr/>
<!-- Granite3 end-->
<!-- L I N K S      S E COND  C OL U M N-->
<a href="http://www.mirror.co.uk/3am/">3 AM GIRLS</a><br/>

<a href="http://www.washingtonexaminer.com/author/byron-york">BYRON YORK</a>
</b></tt>
</td>
END Links second column

Begin Posts Third (Right) column
<td align="center" valign="top" width="3"><div style="width:1px;background-color:#C0C0C0;margin-left:1px;margin-right:1px;height:2500px;"></div></td>
<td align="LEFT" valign="TOP" width="33%"><tt><b>
<a href="https://www.nbcnews.com/news/weather/coldest-air-three-years-coming-parts-country-rcna11618">Coldest air in years coming to USA...</a><br/><br/>

<a href="https://www.dailymail.co.uk/news/article-10384831/Ritzy-San-Francisco-condo-tower-leaning-26-INCHES.html">San Fran tower now leaning 26 inches with tilt set to progress three inches year...</a>
<hr/>
END Posts Third column

Begin Links Third (Right) column
<!-- Granite3 Right -->
<!-- /218243714/DR-right_1 -->
<div id="div-gpt-ad-1564685863820-0">
<script>
googletag.cmd.push(function() { googletag.display('div-gpt-ad-1564685863820-0'); });
</script>
</div>
<div id="div-gpt-ad-1564811295535-0"></div>
<!-- Granite3 End -->
<hr/>
<a href="https://apps.apple.com/us/app/official-drudge-report-free/id375614185">UPDATE:  DRUDGE APP IPHONE, IPAD...</a><br/><br/>


End of Third column 
<a href="/privacy/california/" id="ca_notice">CALIFORNIA NOTICE</a><br/>
<a href="/privacy/opt-out/" id="ca_info">DO NOT SELL MY INFO</a><br/>
<a href="/privacy/">PRIVACY POLICY</a>
<hr/>
</b></tt></td>
</tr>
</table>
<div id="div-204" style="visibility:hidden"></div>
<script>dr_check_consent();</script>
</center></body>
</html>
&lt;!-- Copyright 2021 by Drudge Report --!&gt;   
&lt;!-- END -- DO NOT REMOVE THIS LINE --Adrian --!&gt;


In [18]:
soup = html_soup
head_tag = soup.head
head_tag
# <head><title>The Dormouse's story</title></head>

head_tag.title.text
# 'DRUDGE REPORT 2022®'


'DRUDGE REPORT 2024®'

In [3]:
body_tag = soup.body
# body_tag
# <head><title>The Dormouse's story</title></head>

# body_tag.title.text
# 'DRUDGE REPORT 2022®'


NameError: name 'soup' is not defined

In [20]:
Htag = soup.head
Htag.contents

['\n',
 <meta content="upgrade-insecure-requests" http-equiv="Content-Security-Policy"/>,
 '\n',
 <title>DRUDGE REPORT 2024®</title>,
 '\n',
 ' eProof, DR Headline Updater ',
 '\n',
 <script src="//cdnjs.cloudflare.com/ajax/libs/jquery/3.4.1/jquery.min.js" type="text/javascript"></script>,
 '\n',
 <script src="//eproof.drudgereport.com/dr.js" type="text/javascript"></script>,
 '\n',
 ' eProof.com end ',
 '\n',
 <script src="/js/cookie_filter_1.0.1.js" type="text/javascript"></script>,
 '\n',
 <script type="text/javascript">
 <!-- 
 var dr_do_bt = 1; var img = new Image(), url = "/204.png", container = document.getElementById("div-204");
 img.onload = function () { container.appendChild(img); };
 img.src = url;
 var timer = setInterval("__drudge__321__autoRefresh()", 1000 * 33 * 3);
 function __drudge__321__autoRefresh(){self.location.reload(true);}
 (function () {
     var __oldClearInterval = clearInterval;
 
     clearInterval = function (arg) {
         if(arg == timer) {
          

In [21]:
Btag = soup.body
Btag.contents

['\n',
 <tt><b>
 <img src="//eproof.drudgereport.com/pixel.gif" style="visibility: collapse;"/>
 <center>
 <!-- Granite3 Top Web -->
 <!-- /218243714/DR-center_1 -->
 <div id="div-gpt-ad-1564685541535-0">
 <script>
 googletag.cmd.push(function() { googletag.display('div-gpt-ad-1564685541535-0'); });
 </script>
 </div>
 <div id="div-gpt-ad-1564810986146-0" style="margin-top:16px;"></div>
 <!-- Granite3 end -->
 </center>
 <!-- TOP LEFT STARTS HERE--><tt><b><br/><br/>
 <a href="https://www.cnn.com/2023/06/09/politics/trump-tape-didnt-declassify-secret-information/index.html" nopush=""><font color="red">Ex-president admits on tape he didn't declassify 'secret information'...</font></a>
 <br/>
 <a href="https://www.washingtonpost.com/national-security/2023/06/09/trump-tape-classified-documents/" nopush=""><font color="red"><i>Incriminating statements on Iran papers...</i></font></a><br/>
 <a href="https://www.politico.com/news/2023/06/09/trump-loses-two-lawyers-just-hours-after-being-indic

In [22]:
len(soup.contents)

3

In [24]:
soup.contents[0]
soup.contents[1]
soup.contents[2]

'\r\n<!-- Copyright 2021 by Drudge Report --!>   \r\n<!-- END -- DO NOT REMOVE THIS LINE --Adrian --!>\r\n'

In [25]:
soup.contents[1]

<html>
<head>
<meta content="upgrade-insecure-requests" http-equiv="Content-Security-Policy"/>
<title>DRUDGE REPORT 2024®</title>
<!-- eProof, DR Headline Updater -->
<script src="//cdnjs.cloudflare.com/ajax/libs/jquery/3.4.1/jquery.min.js" type="text/javascript"></script>
<script src="//eproof.drudgereport.com/dr.js" type="text/javascript"></script>
<!-- eProof.com end -->
<script src="/js/cookie_filter_1.0.1.js" type="text/javascript"></script>
<script type="text/javascript">
<!-- 
var dr_do_bt = 1; var img = new Image(), url = "/204.png", container = document.getElementById("div-204");
img.onload = function () { container.appendChild(img); };
img.src = url;
var timer = setInterval("__drudge__321__autoRefresh()", 1000 * 33 * 3);
function __drudge__321__autoRefresh(){self.location.reload(true);}
(function () {
    var __oldClearInterval = clearInterval;

    clearInterval = function (arg) {
        if(arg == timer) {
                console.log("clearInterval of TIMER intercepted! (" 

In [28]:
soup.contents[2]

'\r\n<!-- Copyright 2021 by Drudge Report --!>   \r\n<!-- END -- DO NOT REMOVE THIS LINE --Adrian --!>\r\n'

In [26]:
for string in soup.strings:
    print(repr(string))

' '
'\n'
'\n'
'\n'
'DRUDGE REPORT 2024®'
'\n'
'\n'
'\n'
'\n'
'\n'
'\n'
'\n'
'\n'
'\n'
'\n'
'\n'
'\n'
'\n'
'\n'
'\n'
'\n'
'\n'
'\n'
'\n'
'\n'
'\n'
'\n'
'\n'
'\n'
'\n'
"Ex-president admits on tape he didn't declassify 'secret information'..."
'\n'
'\n'
'Incriminating statements on Iran papers...'
'\n'
'TOP LAWYERS ABRUPTLY RESIGN...'
'\n'
'Aide Seen Moving Boxes Also Hit With Indictment...'
'\n'
"Extraordinary test of Garland's integrity..."
'\n'
'\n'
'\n'
'TRUMP INDICTMENT UNSEALED'
'TEXT'
'CLASSIFIED DOCS IN SHOWER'
'\n'
'REVEALED PENTAGON PLAN OF ATTACK'
'\n'
'37 COUNTS!'
'\n'
'FACES 100 YEARS'
'\n'
'DOWNFALL OF THE DON?'
'\n'
'\n'
'\n'
'\n'
'\n'
'\n'
'\n'
'\n'
'\n'
'\n'
'\n'
'JACK SMITH SPEAKS AT 3 PM EST...'
'\n'
'\n'
'Avoids Legal Fight by Moving Case to Miami...'
'\n'
'\n'
'Inside Bedminster When Call Came...'
'\n'
'Trump Turns to Familiar Playbook to Undermine...'
' '
'\n'
'Violent rhetoric spreading on MAGA sites...'
' '
"The 'Lock HIM Up' Election..."
'\n'
'\n'
'META Reveals TW

In [27]:
# To remove extra whitespace, use .stripped_strings generator −

for string in soup.stripped_strings:
    print(repr(string))

'DRUDGE REPORT 2024®'
"Ex-president admits on tape he didn't declassify 'secret information'..."
'Incriminating statements on Iran papers...'
'TOP LAWYERS ABRUPTLY RESIGN...'
'Aide Seen Moving Boxes Also Hit With Indictment...'
"Extraordinary test of Garland's integrity..."
'TRUMP INDICTMENT UNSEALED'
'TEXT'
'CLASSIFIED DOCS IN SHOWER'
'REVEALED PENTAGON PLAN OF ATTACK'
'37 COUNTS!'
'FACES 100 YEARS'
'DOWNFALL OF THE DON?'
'JACK SMITH SPEAKS AT 3 PM EST...'
'Avoids Legal Fight by Moving Case to Miami...'
'Inside Bedminster When Call Came...'
'Trump Turns to Familiar Playbook to Undermine...'
'Violent rhetoric spreading on MAGA sites...'
"The 'Lock HIM Up' Election..."
'META Reveals TWITTER Competitor Planned as Stand-Alone App...'
"'Platform that is sanely run'..."
'BLUE JAYS cut pitcher after latest anti-LGBTQ comments...'
"NOONAN:  Don't Count Third Party Out in '24..."
'Psychedelic Drug Use Among Young Adults Doubles...'
'Millions still under air quality warnings in US as weather ea

In [28]:
# body_tag = soup.body
foo = body_tag.find_next("TOP LEFT STARTS HERE")

print(foo)

bar = body_tag.find_next("L I N K S      S E COND  C OL U M N")
# <!-- L I N K S      S E COND  C OL U M N-->

print(bar)

None
None


In [39]:
Comment = "TOP LEFT STARTS HERE"
comments = soup.find_all(string=lambda text: isinstance(text, Comment) and Comment in text)
print(comments)

TypeError: isinstance() arg 2 must be a type or tuple of types

In [29]:
Comment = "TOP LEFT STARTS HERE"
comments = soup.findAll(text=lambda text:isinstance(text, Comment))
print(comments)

/tmp/ipykernel_609702/845625993.py:2: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  comments = soup.findAll(text=lambda text:isinstance(text, Comment))


TypeError: isinstance() arg 2 must be a type or tuple of types

In [ ]:
for link in html_soup.find_all('a'):
    # print(link.get('href'))
    print(link.text)


POLL: Inflation up, virus down as priorities in USA...
Dems NOT shutting things down this time...
ENOUGH: More Saying They're 'Vaxxed and Done'...
Boris to launch 'Living with Covid' plan in UK...
Where you are most likely to catch?
Evidence AGAINST mandates piles up...
Infections Set to Triple Pre-Omicron Record...
Desperate hospitals ask sick staff to return...
Beds Cut as Nurses Stay Home...
UPDATE: 'Deltacron' likely result of lab error...
T Cells Triggered by Common Cold Also Fend Off Corona...
In much of West, walls closing in on anti-vaxxers...
Sweden imposing curfew...
Australian judge says Djokovic can stay but saga not over...
Pranksters hijack virtual court hearing to stream porn...
SAGET DEAD IN ORLANDO HOTEL ROOM
CAUSE UNKNOWN
FOUND BY HOUSEKEEPING AFTER MISSING CHECKOUT

Fed Vice Chair out following scrutiny over pandemic trades...
Newsom wants health coverage for all illegals...
STUDY: Meditation Powerful Enough To Fight Cancer, COVID?
Extreme Retreat Alters Genes...
UPD

# Build the Loop

In [ ]:
# find the total number of posts to find the limit of the pagination
find_total = html_soup.find('div', class_= 'search-legend')



# grab the total count of posts 
total_posts = int(find_total.find('span', class_='totalcount').text) 



# vary the value of the page parameters
pages = np.arange(0, total_posts+1, 120)



# vary the value of the page parameters
pages = np.arange(0, total_posts+1, 120)



# count tracker for number of iterations
iterations = 0



# initialize empty lists where we'll store our date 
post_times = []
post_neighborhoods = []
post_titles = []
post_bedrooms = []
post_sqft = []
post_links = []
post_prices = []



# create for loop
for page in pages:
    
    # get request
    response = get("https://boston.craigslist.org/search/apa?" 
                   + "s=" # parameter for defining the page number 
                   + str(page) # page number in the pages array 
                   + "&hasPic=1"
                   + "&availabilityMode=0")
    
    # control the crawl rate 
    sleep(randint(1,10))
    
    # throw warning for status codes that are not 200
    if response.status_code != 200:
        warn('Request: {}; Status code: {}'.format(requests, response.status_code))
        
    # define the html text
    html = BeautifulSoup(response.text, 'html.parser')
    
    # define the posts
    posts = html_soup.find_all('li', class_= 'result-row')
    
    
    # extract data item-wise
    for post in posts:

        # if we aren't missing the neighborhood information
        if post.find('span', class_ = 'result-hood') is not None:

            # date
            post_datetime = post.find('time', class_= 'result-date')['datetime']
            post_times.append(post_datetime)

            # neighborhoods
            post_hoods = post.find('span', class_= 'result-hood').text
            post_neighborhoods.append(post_hoods)

            # title 
            post_title = post.find('a', class_='result-title hdrlnk')
            post_title_text = post_title.text
            post_titles.append(post_title_text)

            # link
            post_link = post_title['href']
            post_links.append(post_link)
            
            # removes the \n whitespace from each side, removes the currency symbol, and turns it into an int
            post_price = int(float(post.a.text.strip().replace("$", "").replace(",","")))
            post_prices.append(post_price)
            
            
            
            
            
            # if the number of bedrooms OR sqft aren't missing 
            if post.find('span', class_ = 'housing') is not None:
                
                # if the first element is accidentally square footage
                if 'ft' in post.find('span', class_ = 'housing').text.split()[0]:
                    
                    # make bedroom NaN
                    bedroom_count = np.nan
                    post_bedrooms.append(bedroom_count)
                    
                    # make sqft the first element
                    sqft = int(post.find('span', class_ = 'housing').text.split()[0][:-3])
                    post_sqft.append(sqft)
                    
                # if the length of the housing details element is more than 2
                elif len(post.find('span', class_ = 'housing').text.split()) > 2:
                    
                    # therefore element 0 will be bedroom count
                    bedroom_count = post.find('span', class_ = 'housing').text.replace("br", "").split()[0]
                    post_bedrooms.append(bedroom_count)
                    
                    # and sqft will be number 3, so set these here and append
                    sqft = int(post.find('span', class_ = 'housing').text.split()[2][:-3])
                    post_sqft.append(sqft)
                    
                # if there is num bedrooms but no sqft
                elif len(post.find('span', class_ = 'housing').text.split()) == 2:
                    
                    # therefore element 0 will be bedroom count
                    bedroom_count = post.find('span', class_ = 'housing').text.replace("br", "").split()[0]
                    post_bedrooms.append(bedroom_count)
                    
                    # and sqft will be number 3, so set these here and append
                    sqft = np.nan
                    post_sqft.append(sqft)                    
                
                else:
                    bedroom_count = np.nan
                    post_bedrooms.append(bedroom_count)
                
                    sqft = np.nan
                    post_sqft.append(sqft)
                
            # if none of those conditions catch, make bedroom NaN 
            else:
                bedroom_count = np.nan
                post_bedrooms.append(bedroom_count)
                
                sqft = np.nan
                post_sqft.append(sqft)
                
    iterations += 1
      
    print("Page " + str(iterations) + " scraped successfully!")

    
print("\n")
print("Scrape complete!")

Page 1 scraped successfully!
Page 2 scraped successfully!
Page 3 scraped successfully!
Page 4 scraped successfully!
Page 5 scraped successfully!
Page 6 scraped successfully!
Page 7 scraped successfully!
Page 8 scraped successfully!
Page 9 scraped successfully!
Page 10 scraped successfully!
Page 11 scraped successfully!
Page 12 scraped successfully!
Page 13 scraped successfully!
Page 14 scraped successfully!
Page 15 scraped successfully!
Page 16 scraped successfully!
Page 17 scraped successfully!
Page 18 scraped successfully!
Page 19 scraped successfully!
Page 20 scraped successfully!
Page 21 scraped successfully!
Page 22 scraped successfully!
Page 23 scraped successfully!
Page 24 scraped successfully!
Page 25 scraped successfully!
Page 26 scraped successfully!


Scrape complete!


In [ ]:
phx_apts = pd.DataFrame({'posted': post_times,
                       'neighborhood': post_neighborhoods,
                       'post title': post_titles,
                       'number bedrooms': post_bedrooms,
                        'sqft': post_sqft,
                        'URL': post_links,
                       'price': post_prices})

print(phx_apts.info())
phx_apts

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1664 entries, 0 to 1663
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   posted           1664 non-null   object 
 1   neighborhood     1664 non-null   object 
 2   post title       1664 non-null   object 
 3   number bedrooms  1560 non-null   object 
 4   sqft             1352 non-null   float64
 5   URL              1664 non-null   object 
 6   price            1664 non-null   int64  
dtypes: float64(1), int64(1), object(5)
memory usage: 91.1+ KB
None


,posted,neighborhood,post title,number bedrooms,sqft,URL,price
0,2021-01-26 19:44,(PHOENIX),"Dog Park, 24 hour Fitness Center, Garages Avai...",2,865.0,https://phoenix.craigslist.org/cph/apa/d/phoen...,1251
1,2021-01-26 19:42,"(5740 North 59th Ave, Glendale, AZ, US)",Make Ranchwood your home TODAY! Apply online f...,1,540.0,https://phoenix.craigslist.org/wvl/apa/d/glend...,888
2,2021-01-26 19:41,"(5740 North 59th Ave, Glendale, AZ, US)",High speed internet included! Make Ranchwood y...,1,676.0,https://phoenix.craigslist.org/wvl/apa/d/glend...,881
3,2021-01-26 19:27,(near Scottsdale Rd / McDowell),Home in Scottsdale,3,NaN,https://phoenix.craigslist.org/evl/apa/d/scott...,1895
4,2021-01-26 19:13,(Phoenix),Luxury studio residence atop Palomar Hotel in ...,1,840.0,https://phoenix.craigslist.org/cph/apa/d/phoen...,800
...,...,...,...,...,...,...,...
1659,2021-01-26 17:55,"(2625 E. Camelback Rd., Phoenix, AZ)","Sit Back, Relax. Your New View Awaits.",2,1074.0,https://phoenix.craigslist.org/evl/apa/d/phoen...,2408
1660,2021-01-26 17:53,"(2300 W Pecos Rd Chandler, AZ)",Come see our beautiful community!!!,1,756.0,https://phoenix.craigslist.org/evl/apa/d/chand...,1352
1661,2021-01-26 17:53,(Phoenix),Two bedroom in Beautiful Ahwatukee,2,1205.0,https://phoenix.craigslist.org/evl/apa/d/phoen...,1590
1662,2021-01-26 17:52,(Mesa),Stay & Play! Fun Amenities! Call Now!,2,828.0,https://phoenix.craigslist.org/evl/apa/d/mesa-...,1185


In [ ]:
# to move all the scraped data to a CSV file

phx_apts.to_csv('phx_apts.csv', index=False)